# Browse Workflows

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
    
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.calculation.work import WorkCalculation
from aiida.work.run import run, async, submit
from aiida.common.links import LinkType

import ipywidgets as ipw
import datetime
from pprint import pprint
from base64 import b64decode
from collections import OrderedDict
from IPython.display import display, clear_output, Image

# needed for resubmission
from apps.surfaces.nanoribbon.nanoribbonwork import NanoribbonWorkChain
from apps.surfaces.slab.slabwork import SlabGeoOptWorkChain

In [ ]:
# preprocessing
qb = QueryBuilder()
qb.append(WorkCalculation, filters={'extras': {'!has_key': 'resubmitted'}})
for n in qb.all():
    n[0].set_extra('resubmitted', False)

qb = QueryBuilder()
qb.append(WorkCalculation, filters={'extras': {'!has_key': 'obsolete'}})
for n in qb.all():
    n[0].set_extra('obsolete', False)

In [ ]:
def on_search_change(c):
    update_search()

age_range = ipw.IntRangeSlider(
                value=[0, 7],
                min=0,
                max=100,
                step=1,
                description='age in days:',
                layout=ipw.Layout(width="900px"))
age_range.observe(on_search_change, names='value')

state_select = ipw.RadioButtons(options=['new', 'succeed', 'failed', 'resubmitted', 'obsolete', 'all'], description='State:')
state_select.observe(on_search_change, names='value')

display(age_range, state_select)

In [ ]:
def update_search():
    wf_select.options = {"Searching...": False}
    
    min_age = datetime.datetime.now() - datetime.timedelta(days=age_range.value[0])
    max_age = datetime.datetime.now() - datetime.timedelta(days=age_range.value[1])

    filters = {}
    filters['ctime'] = {'and':[{'<=': min_age},{'>': max_age}]}
    filters['attributes'] = {'!has_key': 'source_code'} # exclude workfunctions
    
    if state_select.value == 'new':
        filters['extras'] = {'!has_key': 'preprocess_successful'}

    elif state_select.value == 'succeed':
        filters['extras.preprocess_successful'] = True
        filters['extras.obsolete'] = False

    elif state_select.value == 'failed':
        filters['extras.preprocess_successful'] = False
        filters['extras.resubmitted'] = False

    elif state_select.value == r'resubmitted':
        filters['extras.resubmitted'] = True

    elif state_select.value == r'obsolete':
        filters['extras.obsolete'] = True

    else:
        assert state_select.value == 'all'
    
    qb = QueryBuilder()
    qb.append(WorkCalculation, filters=filters)
    qb.order_by({WorkCalculation:{'ctime':'desc'}})

    options = OrderedDict()
    for n in qb.iterall():
        n = n[0]
        ctime = n.ctime.strftime("%Y-%m-%d %H:%M")
        nsteps = len(n.get_outputs())
        label = "PK %d; %s; %s; steps: %d" %(n.pk, ctime, n.get_attr("_process_label"), nsteps)
        if "structure" in n.get_inputs_dict():
            s = n.inp.structure
            label += "   structure: PK: %s %s"%(s.pk, s.description)
        options[label] = n
    wf_select.options = options
    wf_select.value = options.values()[0] if options else None

In [ ]:
def on_wf_changed(c):
    update_wf()

wf_select = ipw.Select(layout=ipw.Layout(height="300px", width="900px"))
wf_select.observe(on_wf_changed, names='value')
display(wf_select)

In [ ]:
def on_kill_clicked(b):
    kill_wf(wf_select.value)

def on_resub_clicked(b):
    resubmit_wf(wf_select.value)
    update_search()

def on_toggle_resubmitted_clicked(b):
    state = wf_select.value.get_extra('resubmitted')
    wf_select.value.set_extra('resubmitted', not state)
    update_search()

def on_toggle_obsolete_clicked(b):
    state = wf_select.value.get_extra('obsolete')
    wf_select.value.set_extra('obsolete', not state)
    update_search()

def on_refresh_clicked(b):
    update_search()

btn_kill = ipw.Button(description="Kill Workflow")
btn_kill.on_click(on_kill_clicked)

btn_resub = ipw.Button(description="Resubmit Workflow")
btn_resub.on_click(on_resub_clicked)

btn_toggle_resub = ipw.Button(description="Toggle resubmitted")
btn_toggle_resub.on_click(on_toggle_resubmitted_clicked)

btn_toggle_obsolete = ipw.Button(description="Toggle obsolete")
btn_toggle_obsolete.on_click(on_toggle_obsolete_clicked)

refresh_btn = ipw.Button(description="Refresh")
refresh_btn.on_click(on_refresh_clicked)

btn_box = ipw.HBox([btn_kill, btn_resub, btn_toggle_resub, btn_toggle_obsolete, refresh_btn])
display(btn_box)

In [ ]:
progress = ipw.IntProgress(description="Loading...", max=100)
display(progress)

In [ ]:
wf_out = ipw.Output(layout=ipw.Layout(border="2px solid black"))
display(wf_out)

In [ ]:
def update_wf():
    with wf_out:
        clear_output()
        btn_box.layout.display = 'none' # hide buttons
            
        node = wf_select.value
        if not node:
            print("no selection")
            return

        progress.value = 1
        progress.layout.display = None # show progress bar
        
        print("NODE:")
        print("  PK: %s"%node.pk)
        print("  Class: %s"%node.get_attr('_process_label'))

        if "thumbnail" in node.get_extras():
            print("\n\nSTRUCTURE:")
            display(Image(data=b64decode(node.get_extra("thumbnail"))))
        progress.value = 5
                
        print("\n\nINPUTS:")
        for k, v in node.get_inputs_dict().items():
            print("  %s: %s"%(k,v))
        
        print("\n\nEXTRAS:")
        for k, v in node.get_extras().items():
            if k != "thumbnail":
                print("  %s: %s"%(k,v))
        progress.value = 10
        
        print("\n\nREPORT:")
        ! verdi work report $node.pk
        progress.value = 40
        
        print("\n\nTREE:")
        ! verdi work tree --node-label label $node.pk
        progress.value = 70
        
        print("\n\nSTATE:")
        ! verdi work tree --node-label state $node.pk
        progress.value = 100
        progress.layout.display = 'none' # hide progress bar
        
        # enable buttons if applicable
        failed = not node.get_extra("preprocess_successful", False)
        btn_toggle_resub.disabled = not failed
        btn_resub.disabled = not failed
        btn_kill.disabled = node.is_sealed
        btn_box.layout.display = None # show buttons

In [ ]:
def kill_wf(wf):
    if not wf.is_sealed:
        wf._set_attr('_aborted', True)

    for child in wf.get_outputs(link_type=LinkType.CALL):
        if not child.has_finished():
            print("Killing: PK=%d"%child.pk)
            child.kill()

In [ ]:
def resubmit_wf(wf):
    wfclass = eval(wf.get_attr('_process_label')) # requires prev import of class
    wfinps = wf.get_inputs_dict()
    
    outputs = submit(wfclass, **wfinps)
    print(outputs) 
    wf.set_extra('resubmitted', True)

In [ ]:
update_search()